# Generate the footy stats <a class="tocSkip">

## Import required libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

## Set indexes
Indexes will bind to be columns of dataframe

In [2]:
data_index = ["BindKey",  
              "Kicks", "Marks", "Handballs", "Disposals", "Goals", "Behinds", "HitOuts",
              "Tackles", "Rebound50s", "Inside50s", "Clearances", "Clangers", 
              "FreesFor", "FreesAgainst", "BrownlowVotes", 
              "ContestedPossesions", "UncontestedPossesions", 
              "ContestedMarks", "MarksInside50s",  "OnePercenters", "Bounces",
              "GoalAssist"]

metadata_index = ["Date", "Team_home", "Team_away", 
                  "Quarters_home", "Quarters_away", 
                  "Total_home", "Total_away"]

## Define match data functions

In [3]:
def get_match_data(url):
    """
    Match data taken by url and first two tables in the url.
    Returns a pandas series of the match data for a given game.
    """
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    tables = soup.findAll("table", attrs={'class':"sortable"})[:2]

    # Create the game data frame and return as a series
    return pd.merge(*[# Create pandas series of the ‘totals’ row binding totals as the key
                      pd.Series([cell.text.strip() 
                                 for cell in table_row.findAll('td')[:-1]], 
                                 index=data_index).to_frame().transpose().set_index("BindKey")
                      # Do this for home and away team
                      for table in tables
                      # Iterate through each row of the table
                      for table_row in table.select("tr")
                      # But make sure we do this only for the last row
                      # List comprehensions can make order confusing sometimes
                      if len(table_row.findAll('td')) > 0 and 
                         table_row.findAll('td')[0].text == "Totals"],
                      # Parameters of pd_concat set first table to home side.
                      # And second table to the away side. 
                      # Then drop the bind key and return as series
                      left_index=True, right_index=True,
                      suffixes=["_home", "_away"]).reset_index(drop=True).squeeze()

def get_match_metadata(table):
    """
    Get metadata for a given game
    Also gets the match data whilst at it.
    Returns the metadata and match data as pandas series.
    """
    home, away = table.select("tr")
    Team_home, Quarters_home, Total_home, Date = [td.text 
                                                  for td in home.findAll("td")]
    Date = ' '.join(Date.split(" ", 4)[:-1])
    Team_away, Quarters_away, Total_away, na = [td.text 
                                                for td in away.findAll("td")]
    metadata_series = pd.Series([Date, Team_home, Team_away, 
                                 Quarters_home, Quarters_away, 
                                 Total_home, Total_away], 
                                index=metadata_index)
    # Get data for game.
    data_url = os.path.join("https://afltables.com/afl/seas/", away.findAll("td")[-1].contents[-2]['href'])
    data_table = get_match_data(data_url)
    # Merge the two dataframes.
    return pd.concat([metadata_series, data_table], axis='rows')

## Get data from afl tables

Get all the tables that contain match stats, these are the ones with the games.  
Remove the last nine games as we just want the home and away season.

In [6]:
# Initalise website to query from
r = requests.get("https://afltables.com/afl/seas/2017.html")
soup = BeautifulSoup(r.text, "html.parser")

# Get list of games
games = [game 
          for game in soup.findAll("table", attrs={'border':'1', 'style':"font: 12px Verdana;"})
          if "Match stats" in game.text][:-9]  # Last 9 games are finals

# Generate a dataframe, obtaining the metadata, and subsquently the match data.
df = pd.concat([get_match_metadata(game)
                for game in games],
               axis='columns').transpose()

# Convert date column to standard datetime format
df['Date'] = pd.to_datetime(df['Date'])

## Write data to csv file
Result is a dataframe with each row representing a match 
and the column representing a stat of the game.

In [9]:
csv_path = os.path.join(os.path.dirname(os.path.realpath("__file__")), os.pardir, "data", "stats.2017.csv")
df.to_csv(csv_path, sep=",", index=False, header=True)